In [ ]:
import os
import pytz

import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nfl_data_py as nfl

np.float_ = np.float64


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

In [ ]:
qb_cols = ['player_display_name','player_id','position','season','week','season_type','passing_epa','rushing_epa']
roster_cols = ['player_id','season','years_exp','draft_number']
years = range(1999, 2024)

if not os.path.exists('../data/qb_epa.pkl'):
    df_qb = nfl.import_weekly_data(years, columns=qb_cols).query('(position=="QB")')

    # Merge draft number and years of experience
    df_player = nfl.import_seasonal_rosters(years,columns=roster_cols)
    df_qb = df_qb.merge(df_player, how='left', on=['player_id','season'])

    # Impute missing epa values
    df_qb.loc[:,['passing_epa','rushing_epa']] = df_qb.loc[:,['passing_epa','rushing_epa']].fillna(0, axis=1)

    # Impute missing draft numbers
    df_qb['draft_number'] = df_qb.draft_number.replace({None: np.nan}).astype(float)
    df_qb['draft_number'] = df_qb.groupby('player_id')['draft_number'].transform(lambda x: x.fillna(x.mean()))

    df_qb['draft_number'] = df_qb['draft_number'].fillna(300)

    df_qb.to_pickle('../data/qb_epa.pkl')
else:
    df_qb = pd.read_pickle('../data/qb_epa.pkl')

df_qb.head()

In [ ]:
df_qb['total_epa'] = df_qb['passing_epa'] + df_qb['rushing_epa']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

max_games = 120

# Use ggplot style
plt.style.use('ggplot')

# update fig size
plt.rcParams['figure.figsize'] = [10, 6]

# List of players to include in the plot
# players = ['Patrick Mahomes', 'Josh Allen', 'Lamar Jackson', 'Joe Burrow', 'Justin Herbert', 'Dak Prescott', 'Jalen Hurts', 'Aaron Rodgers', 'Tom Brady', 'Drew Brees', 'Peyton Manning', 'Russell Wilson']

players = ['Josh Allen', 'Lamar Jackson', 'Joe Burrow', 'Justin Herbert', 'Dak Prescott', 'Jalen Hurts', 'C.J. Stroud', 'Jordan Love', 'Trevor Lawrence', 'Kyler Murray', 'Tua Tagovailoa']

players = ['Josh Allen', 'Lamar Jackson', 'Joe Burrow', 'Justin Herbert', 'Dak Prescott', 'Jalen Hurts']


# players = ['Patrick Mahomes', 'Aaron Rodgers', 'Tom Brady', 'Peyton Manning']


# players = ['Josh Allen', 'Lamar Jackson', 'Joe Burrow', 'Justin Herbert', 'Dak Prescott', 'Jalen Hurts']

# Dictionary of custom colors for each player
player_colors = {
    'Patrick Mahomes': '#E31837',
    'Josh Allen': 'blue',
    'Lamar Jackson': 'purple',
    'Joe Burrow': 'darkorange',
    'Justin Herbert': 'deepskyblue',
    'Dak Prescott': 'grey',
    'Jalen Hurts': 'seagreen',
    'Aaron Rodgers': 'darkgreen',
    'Tom Brady': 'red',
    'Drew Brees': 'gold',
    'Peyton Manning': 'blue',
    'Russell Wilson': 'green',
    'C.J. Stroud': 'red',
    'Jordan Love': 'green',
    'Trevor Lawrence': 'blue',
    'Kyler Murray': 'red',
    'Tua Tagovailoa': 'green'
}

# Initialize a dictionary to store data for each player
player_data = {}

for player in players:
    # Filter data for the current player
    df_player = df_qb[df_qb['player_display_name'] == player]

    # Sort by season and week
    df_player = df_player.sort_values(by=['season', 'week'], ignore_index=True)

    # Add games played column
    df_player['games_played'] = df_player.index + 1

    # Calculate the cumulative EPA
    df_player['cumulative_epa'] = df_player['total_epa'].cumsum()

    df_player = df_player.query('games_played <= @max_games')

    # Store the processed data in the dictionary
    player_data[player] = df_player

# Plotting and creating the GIF
fig, ax = plt.subplots()
ax.set_xlim(1, max(df['games_played'].max() + 1 for df in player_data.values()))
ax.set_ylim(min(df['cumulative_epa'].min() for df in player_data.values()),
            max(df['cumulative_epa'].max() + 10 for df in player_data.values()))
ax.set_xlabel('Games Played', color='black')
ax.set_ylabel('Cumulative EPA', color='black')
ax.set_title('QB EPA Progression Over Time', color='black')

# Initialize a dictionary to store lines for each player
lines = {}

# Create a line for each player with the specified color
for player in players:
    line, = ax.plot([], [], color=player_colors[player], lw=2, label=player)
    lines[player] = line

# Initialize the plot with empty data
def init():
    for line in lines.values():
        line.set_data([], [])
    return list(lines.values())

# Update function for the animation
def update(frame):
    for player in players:
        week_data = player_data[player].iloc[:frame+1]
        lines[player].set_data(week_data['games_played'], week_data['cumulative_epa'])
    return list(lines.values())

# Create animation
ani = animation.FuncAnimation(
    fig, update, frames=len(max(player_data.values(), key=len)), init_func=init, blit=True, repeat=False
)

# Add a legend
ax.legend(loc='upper left')

# add this line to the bottom right of the image "data @nfl_data_py | x/twitter @lukeneuendorf"
ax.text(1, -0.12, 'data @nfl_data_py', transform=ax.transAxes, ha='right', color='black', fontsize=7)

# Save as GIF
ani.save('cumulative_epa_comparison_custom_colors.gif', writer='imagemagick')

plt.show()